## Demo : Pausing and Resuming a vcluster

### Create vcluster

In [1]:
! vcluster create vc1 -n host-namespace --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/1014842955 --namespace host-namespace --install --repository-config='' --values /tmp/3444456695
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [20]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-7n5zw-x-kube-system-x-vc1   1/1     Running   0          33s
pod/vc1-0                                          2/2     Running   0          50s

NAME                                         TYPE           CLUSTER-IP      EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1         ClusterIP      10.96.148.49    <none>           53/UDP,53/TCP,9153/TCP   33s
service/vc1                                  LoadBalancer   10.96.249.179   172.18.255.201   443:31983/TCP            50s
service/vc1-headless                         ClusterIP      None            <none>           443/TCP                  50s
service/vc1-node-kcd-vcluster-demo-worker2   ClusterIP      10.96.20.31     <none>           10250/TCP                33s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     50s


##### Accessing Vcluster

In [21]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


##### Create a workload inside the vcluster

In [22]:
! kubectl create ns ns1 --kubeconfig ./kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


In [24]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-cczkn   1/1     Running   0          6s
sample-7f75ccc884-6kwtd   1/1     Running   0          6s


##### Pausing the vcluster

In [25]:
 ! vcluster pause vc1 -n host-namespace

[info]   Scale down statefulSet host-namespace/vc1...
[info]   Delete 3 vcluster workloads
[done] √ Successfully paused vcluster host-namespace/vc1


In [26]:
 ! kubectl get pods -n host-namespace

No resources found in host-namespace namespace.


In [27]:
! kubectl get pv,pvc -n host-namespace

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                       STORAGECLASS   REASON   AGE
persistentvolume/pvc-d72ac67b-b74c-4de3-a31e-4dade1411b68   5Gi        RWO            Delete           Bound    host-namespace/data-vc1-0   standard                118s

NAME                               STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
persistentvolumeclaim/data-vc1-0   Bound    pvc-d72ac67b-b74c-4de3-a31e-4dade1411b68   5Gi        RWO            standard       2m2s


##### Resuming the vcluster

In [28]:
! vcluster resume vc1 -n host-namespace

[done] √ Successfully resumed vcluster vc1 in namespace host-namespace


In [30]:
! kubectl get pods -n host-namespace

NAME                                           READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-7n5zw-x-kube-system-x-vc1   1/1     Running   0          13s
sample-7f75ccc884-6kwtd-x-ns1-x-vc1            1/1     Running   0          13s
sample-7f75ccc884-cczkn-x-ns1-x-vc1            1/1     Running   0          13s
vc1-0                                          2/2     Running   0          21s


In [31]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-cczkn   1/1     Running   0          100s
sample-7f75ccc884-6kwtd   1/1     Running   0          100s


### Cleanup

In [32]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/1024023222 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [33]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   42m
kube-node-lease      Active   42m
kube-public          Active   42m
kube-system          Active   42m
local-path-storage   Active   42m
metallb-system       Active   41m
